## 文件准备
- 下载 [gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz](https://azureopendatastorage.blob.core.windows.net/gnomad/release/2.1.1/constraint/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz)  并解压获取文件 gnomad.v2.1.1.lof_metrics.by_gene.txt

- 下载 [gnomAD_v2.1.1_transcripts_with_rmc.tsv](https://storage.googleapis.com/gcp-public-data--gnomad/release/2.1.1/regional_missense_constraint/gnomAD_v2.1.1_transcripts_with_rmc.tsv)

- 提前准备基因名称和基因ID对应文件 "GeneSymbol2ID.tsv",title格式如下
    ```tsv
    Gene_symble	geneID
    A1BG	1
    A1CF	29974
    ...
    ```



In [ ]:
import pandas as pd

In [ ]:
gnomAD_lof_metrics = "gnomad.v2.1.1.lof_metrics.by_gene.txt"
transcripts_with_rmc = "gnomAD_v2.1.1_transcripts_with_rmc.tsv"
GeneSymbol2ID = "GeneSymbol2ID.tsv"

In [ ]:
#db_symbol2ID=pd.read_csv("id_symbol_GRCh37.txt",sep="\t",dtype={"geneID":str})
db_symbol2ID=pd.read_csv(GeneSymbol2ID,sep="\t",dtype={"geneID":str})
db_symbol2ID

,Gene_symble,geneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A2MP1,3
...,...,...
23887,BLACE,338436
23888,FAM27E2,100289124
23889,SLC25A21-AS1,100129794
23890,DHRS4-AS1,55449


In [ ]:
db_lof=pd.read_csv(gnomAD_lof_metrics,sep="\t")
db_lof_use_columns=["chromosome","start_position","end_position","oe_mis","mis_z","gene"]
db_lof_use=db_lof[db_lof_use_columns]
db_lof_use_addID = db_lof_use.merge(db_symbol2ID,left_on="gene",right_on="Gene_symble",how="left")
db_lof_use_addID

,chromosome,start_position,end_position,oe_mis,mis_z,gene,Gene_symble,geneID
0,17,60019966,60142643,0.77921,2.623200,MED13,MED13,9969
1,5,36876861,37066515,0.58688,5.573700,NIPBL,NIPBL,25836
2,10,112327449,112364394,0.28251,6.399900,SMC3,SMC3,9126
3,16,58553855,58663790,0.43290,7.254600,CNOT1,CNOT1,23019
4,1,40627045,40706593,0.68766,3.462000,RLF,RLF,6018
...,...,...,...,...,...,...,...,...
19710,19,9212945,9213982,0.97723,0.108880,OR7G2,OR7G2,390882
19711,19,9236688,9237626,1.00050,-0.002281,OR7G3,OR7G3,390883
19712,11,124179708,124180733,1.09850,-0.445520,OR8D1,OR8D1,283159
19713,19,53267448,53290044,1.25750,-1.731200,ZNF600,ZNF600,162966


In [22]:
db_lof_for_oe = db_lof_use_addID[["chromosome","start_position","end_position","oe_mis","gene","geneID"]]
db_lof_for_oe

,chromosome,start_position,end_position,oe_mis,gene,geneID
0,17,60019966,60142643,0.77921,MED13,9969
1,5,36876861,37066515,0.58688,NIPBL,25836
2,10,112327449,112364394,0.28251,SMC3,9126
3,16,58553855,58663790,0.43290,CNOT1,23019
4,1,40627045,40706593,0.68766,RLF,6018
...,...,...,...,...,...,...
19710,19,9212945,9213982,0.97723,OR7G2,390882
19711,19,9236688,9237626,1.00050,OR7G3,390883
19712,11,124179708,124180733,1.09850,OR8D1,283159
19713,19,53267448,53290044,1.25750,ZNF600,162966


In [23]:
db_PP2 = db_lof_use_addID[["gene","geneID","mis_z"]]
# 保存z-score >= 3.09的基因，并更新df的header
db_PP2[db_PP2.mis_z >= 3.09].rename(columns={"gene":"HGNC","geneID":"entrez_id"}).to_csv("PP2.gene.tsv",sep="\t",index=False)


In [24]:
def get_chrom_start_end(item: pd.Series):
    """
    Returns the region information for a given item.
    """
    chrom,start=item['start_coordinate'].split(':')
    chrom,end = item['stop_coordinate'].split(':')
    return chrom , start, end


In [ ]:
db_with_rmc = pd.read_csv(transcripts_with_rmc,sep="\t")
db_with_rmc["chromosome"]=db_with_rmc.apply(lambda x: x['start_coordinate'].split(':')[0],axis=1)
db_with_rmc["start_position"]=db_with_rmc.apply(lambda x:  x['start_coordinate'].split(':')[1],axis=1)
db_with_rmc["end_position"]=db_with_rmc.apply(lambda x:  x['stop_coordinate'].split(':')[1],axis=1)
db_with_rmc_use_addID = db_with_rmc.merge(db_symbol2ID,left_on="gene_name",right_on="Gene_symble",how="left").rename(columns={"gene_name":"gene","oe":"oe_mis"})

use_columns=["chromosome","start_position","end_position","oe_mis","gene","geneID"]
oe_from_rmc = db_with_rmc_use_addID[use_columns]
oe_from_rmc

,chromosome,start_position,end_position,oe_mis,gene,geneID
0,7,127230150,127231350,0.61849,ARF5,381
1,7,127228553,127230149,0.11729,ARF5,381
2,12,2904306,2906999,0.47272,FKBP4,2288
3,12,2907000,2912421,0.88654,FKBP4,2288
4,3,50223750,50225545,0.91022,SEMA3F,6405
...,...,...,...,...,...,...
17843,21,38997652,39087301,0.14740,KCNJ6,3763
17844,21,39087302,39212984,0.85129,KCNJ6,3763
17845,12,54447707,54448696,0.97567,HOXC4,3221
17846,12,54448697,54448863,0.31853,HOXC4,3221


In [26]:
oe_from_db_lof = db_lof_for_oe[~db_lof_for_oe['gene'].isin(db_with_rmc_use_addID['gene'])]
oe_from_db_lof

,chromosome,start_position,end_position,oe_mis,gene,geneID
22,19,47567444,47617009,0.93465,ZC3H4,23211
48,16,3775055,3930727,0.70864,CREBBP,1387
50,7,2393721,2420380,0.75396,EIF3B,8662
53,1,150293925,150325671,0.43524,PRPF3,9129
55,19,36208921,36229779,0.75854,KMT2B,9757
...,...,...,...,...,...,...
19710,19,9212945,9213982,0.97723,OR7G2,390882
19711,19,9236688,9237626,1.00050,OR7G3,390883
19712,11,124179708,124180733,1.09850,OR8D1,283159
19713,19,53267448,53290044,1.25750,ZNF600,162966


In [ ]:
oe_all = pd.concat([oe_from_rmc, oe_from_db_lof])
oe_all["geneID:oe_mis"]= oe_all["geneID"] + "_" + oe_all["oe_mis"].astype(str)
oe_out_columns=["chromosome","start_position","end_position","geneID:oe_mis","oe_mis","gene","geneID"]

oe_all[oe_out_columns].to_csv("gnomAD.mis_oe.bed",sep="\t",header=False,index=False)

执行如下命令构建成压缩的bed文件并建立索引
```shell
sort -k1,1 -k2,2n -k3,3n gnomAD.mis_oe.bed > gnomAD.mis_oe.sort.bed

bgzip gnomAD.mis_oe.sort.bed

tabix gnomAD.mis_oe.sort.bed.gz
```

生成的 `PP2.gene.tsv` 用于ACMG模块库文件中

生成的 `gnomAD.mis_oe.sort.bed.gz` 用于vep注释的custom中,注释后结果字段为 `entrezID_oe`

```
--custom file=/ifstj2/B2C_RD_H1/Personal/liubo/4.database/gnormAD/v2.1.1/gnomAD.mis_oe.sort.bed.gz,short_name=entrezID_oe,format=bed,type=overlap
```